### THE GENERATED CODE BY LLM

In [2]:
from z3 import Solver, Real, sat

def calc_value_added_tax(output_tax_val: float, input_tax_val: float):
    s = Solver()

    output_tax_amount = Real('output_tax_amount')
    input_tax_amount  = Real('input_tax_amount')
    final_tax_amount  = Real('final_tax_amount')

    # 建立 constraints
    s.add(output_tax_amount == output_tax_val)
    s.add(input_tax_amount  == input_tax_val)
    s.add(final_tax_amount  == output_tax_amount - input_tax_amount)

    # 用 z3.sat 做檢查
    if s.check() == sat:
        m = s.model()
        # 先取得解，轉換成 float，再四捨五入至整數
        tax_value = float(m[final_tax_amount].as_decimal(20))
        return int(round(tax_value))
    else:
        raise ValueError("Solver 無法求解或約束衝突。")


def calc_non_value_added_tax(sales_val: float, category: int):
    """
    依照「非加值型營業稅」邏輯計算應納稅額，
    並以四捨五入至整數後回傳。
    
    參數：
      sales_val: 銷售額
      category:  傳入 1~8 的類別代號，對應不同稅率 (TAX_RATE_MAP)
        1.小規模營業人及其他經財政部規定免予申報銷售額之營業人：1%
        2.保險業之再保費收入：1%
        3.銀行業、保險業、信託投資業、證券業、期貨業、票券業及典當業經營專屬本業之銷售額之收入（不含銀行業、保險業經營銀行、保險本業收入）：2%
        4.銀行業、保險業經營銀行、保險本業之收入：5%
        5.銀行業、保險業、信託投資業、證券業、期貨業、票券業及典當業經營非專屬本業之銷售額之收入：5%
        6.夜總會、有娛樂節目之餐飲店：15%
        7.酒家及有陪侍服務之茶室、咖啡廳、酒吧等：25%
        8.農產品批發市場之承銷人及銷售農產品之小規模營業人：0.1%
      
    回傳：
      final_tax_amount (int)：應納營業稅額（四捨五入後）
    """
    TAX_RATE_MAP = {
        1: 0.01,   # 1%
        2: 0.01,   # 1%
        3: 0.02,   # 2%
        4: 0.05,   # 5%
        5: 0.05,   # 5%
        6: 0.15,   # 15%
        7: 0.25,   # 25%
        8: 0.001   # 0.1% => 0.001
    }
    if category not in TAX_RATE_MAP:
        raise ValueError(f"類別代號 {category} 不在稅率對應表中，請確認輸入是否正確。")
    tax_rate_val = TAX_RATE_MAP[category]

    s = Solver()

    sales_amount = Real('sales_amount')
    tax_rate = Real('tax_rate')
    final_tax_amount = Real('final_tax_amount')

    s.add(sales_amount == sales_val)
    s.add(tax_rate == tax_rate_val)
    s.add(final_tax_amount == sales_amount * tax_rate)

    if s.check() == sat:
        m = s.model()
        tax_value = float(m[final_tax_amount].as_decimal(20))
        return int(round(tax_value))
    else:
        raise ValueError("Solver 無法求解或約束衝突。")


if __name__ == "__main__":
    # ============ 範例 1：加值型營業稅 =============
    # 範例：銷項稅額 = 15000 * 5% = 750 元
    #       進項稅額 = 10000 * 5% = 500 元
    #       應納稅額 = 750 - 500 = 250 元
    vat_example = calc_value_added_tax(output_tax_val=750, input_tax_val=500)
    print(f"[加值型營業稅] 應納(溢付)稅額：{vat_example} 元")

    # 以第 1 類為例 (小規模營業人，稅率=1%)，銷售額=100000
    category_1_example = calc_non_value_added_tax(sales_val=100000, category=1)
    print(f"[非加值型營業稅] 第1類, 銷售額=100000, 應納稅額: {category_1_example} 元")

    # 以第 8 類為例 (農產品批發市場之承銷人 & 小規模營業人，稅率=0.1%)，銷售額=100000
    category_8_example = calc_non_value_added_tax(sales_val=100000, category=8)
    print(f"[非加值型營業稅] 第8類, 銷售額=100000, 應納稅額: {category_8_example} 元")

[加值型營業稅] 應納(溢付)稅額：250 元
[非加值型營業稅] 第1類, 銷售額=100000, 應納稅額: 1000 元
[非加值型營業稅] 第8類, 銷售額=100000, 應納稅額: 100 元


### Selenium Test

In [5]:
import random
import time
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from z3 import Solver, Real, sat

def round_half_up(x: float) -> int:
    """
    正常四捨五入函式：
      當小數部分 >= 0.5 時，往上取整，
      適用於非負數情形（稅額通常為正）。
    """
    return int(math.floor(x + 0.5))

def calc_value_added_tax(output_tax_val: float, input_tax_val: float):
    s = Solver()

    output_tax_amount = Real('output_tax_amount')
    input_tax_amount  = Real('input_tax_amount')
    final_tax_amount  = Real('final_tax_amount')

    # 建立 constraints
    s.add(output_tax_amount == output_tax_val)
    s.add(input_tax_amount  == input_tax_val)
    s.add(final_tax_amount  == output_tax_amount - input_tax_amount)

    # 用 z3.sat 做檢查
    if s.check() == sat:
        m = s.model()
        tax_value = float(m[final_tax_amount].as_decimal(20))
        # 使用自訂的正常四捨五入函式
        return round_half_up(tax_value)
    else:
        raise ValueError("Solver 無法求解或約束衝突。")


def calc_non_value_added_tax(sales_val: float, category: int):
    """
    依照「非加值型營業稅」邏輯計算應納稅額，
    並以正常四捨五入（xxxx.5時進位）後回傳整數結果。
    
    參數：
      sales_val: 銷售額
      category:  傳入 1~8 的類別代號，對應不同稅率 (TAX_RATE_MAP)
    """
    TAX_RATE_MAP = {
        1: 0.01,   # 1%
        2: 0.01,   # 1%
        3: 0.02,   # 2%
        4: 0.05,   # 5%
        5: 0.05,   # 5%
        6: 0.15,   # 15%
        7: 0.25,   # 25%
        8: 0.001   # 0.1% => 0.001
    }
    if category not in TAX_RATE_MAP:
        raise ValueError(f"類別代號 {category} 不在稅率對應表中，請確認輸入是否正確。")
    tax_rate_val = TAX_RATE_MAP[category]

    s = Solver()

    sales_amount = Real('sales_amount')
    tax_rate = Real('tax_rate')
    final_tax_amount = Real('final_tax_amount')

    s.add(sales_amount == sales_val)
    s.add(tax_rate == tax_rate_val)
    s.add(final_tax_amount == sales_amount * tax_rate)

    if s.check() == sat:
        m = s.model()
        tax_value = float(m[final_tax_amount].as_decimal(20))
        return round_half_up(tax_value)
    else:
        raise ValueError("Solver 無法求解或約束衝突。")


def run_test():
    driver = webdriver.Chrome()
    # 請將此 URL 修改為您實際表單所在的網址
    driver.get("https://www.etax.nat.gov.tw/etwmain/etw158w/40")
    
    # 取得兩個「計算」按鈕（假設網頁上有兩個，分別對應加值型與非加值型）
    calc_buttons = driver.find_elements(By.XPATH, "//button[@title='計算']")
    if len(calc_buttons) < 2:
        print("無法取得足夠的計算按鈕，請確認網頁表單的設定。")
        driver.quit()
        return

    total_tests = 0
    match_count = 0

    for i in range(100):
        print(f"===== 迴圈第 {i+1} 次 =====")
        # -------- 加值型營業稅測試 --------
        output_tax_val = int(round(random.uniform(100, 10000)))
        input_tax_val = int(round(random.uniform(100, 10000)))
        try:
            local_vat = calc_value_added_tax(output_tax_val, input_tax_val)
        except Exception as e:
            print(f"Local calc error (加值型): {e}")
            local_vat = None
        local_vat_int = local_vat if local_vat is not None else None

        try:
            output_tax_elem = driver.find_element(By.ID, "outputTax")
            vat_elem = driver.find_element(By.ID, "vat")
            output_tax_elem.clear()
            output_tax_elem.send_keys(str(output_tax_val))
            vat_elem.clear()
            vat_elem.send_keys(str(input_tax_val))
        except Exception as e:
            print(f"Form fill error (加值型): {e}")
        
        calc_buttons[0].click()
        time.sleep(1)
        try:
            result_spans = driver.find_elements(By.CSS_SELECTOR, "span.ml-2.text-danger")
            if len(result_spans) < 1:
                raise Exception("找不到加值型計算結果的 span 元素")
            web_vat_str = result_spans[0].text.replace(",", "").strip()
            web_vat = float(web_vat_str)
            web_vat_int = round_half_up(web_vat)
        except Exception as e:
            print(f"加值型結果取得失敗: {e}")
            web_vat_int = None

        total_tests += 1
        if local_vat_int is not None and web_vat_int is not None and local_vat_int == web_vat_int:
            match_count += 1
            print(f"Test {i+1} (加值型): Local Tax = {local_vat_int}, Online Tax = {web_vat_int}")
        else:
            print(f"Test {i+1} (加值型): Local Tax = {local_vat_int}, Online Tax = {web_vat_int}")
            print(f"  --> Mismatch! Parameters: {{'output_tax_val': {output_tax_val}, 'input_tax_val': {input_tax_val}}}")
        
        # -------- 非加值型營業稅測試 --------
        sales_val = int(round(random.uniform(10000, 1000000)))
        category = random.randint(1, 8)
        try:
            local_nonvat = calc_non_value_added_tax(sales_val, category)
        except Exception as e:
            print(f"Local calc error (非加值型): {e}")
            local_nonvat = None
        local_nonvat_int = local_nonvat if local_nonvat is not None else None

        try:
            sales_elem = driver.find_element(By.ID, "sales")
            sales_elem.clear()
            sales_elem.send_keys(str(sales_val))
        except Exception as e:
            print(f"Form fill error (非加值型 - sales): {e}")
        
        try:
            select_elem = driver.find_element(By.CSS_SELECTOR, "select[formcontrolname='taxRate']")
            select_obj = Select(select_elem)
            select_obj.select_by_index(category)  # 假設 index 1 對應類別 1
        except Exception as e:
            print(f"Form fill error (非加值型 - select): {e}")
        
        calc_buttons[1].click()
        time.sleep(1)
        try:
            result_spans = driver.find_elements(By.CSS_SELECTOR, "span.ml-2.text-danger")
            if len(result_spans) < 2:
                raise Exception("找不到非加值型計算結果的 span 元素")
            web_nonvat_str = result_spans[1].text.replace(",", "").strip()
            web_nonvat = float(web_nonvat_str)
            web_nonvat_int = round_half_up(web_nonvat)
        except Exception as e:
            print(f"非加值型結果取得失敗: {e}")
            web_nonvat_int = None

        total_tests += 1
        if local_nonvat_int is not None and web_nonvat_int is not None and local_nonvat_int == web_nonvat_int:
            match_count += 1
            print(f"Test {i+1} (非加值型): Local Tax = {local_nonvat_int}, Online Tax = {web_nonvat_int}")
        else:
            print(f"Test {i+1} (非加值型): Local Tax = {local_nonvat_int}, Online Tax = {web_nonvat_int}")
            print(f"  --> Mismatch! Parameters: {{'sales_val': {sales_val}, 'category': {category}}}")
        
        print("\n")
    
    match_percentage = (match_count / total_tests) * 100 if total_tests > 0 else 0
    print(f"總共匹配次數: {match_count} / {total_tests}  ({match_percentage:.2f}%)")
    driver.quit()


if __name__ == "__main__":
    run_test()


===== 迴圈第 1 次 =====
Test 1 (加值型): Local Tax = 1663, Online Tax = 1663
Test 1 (非加值型): Local Tax = 3664, Online Tax = 3664


===== 迴圈第 2 次 =====
Test 2 (加值型): Local Tax = 3227, Online Tax = 3227
Test 2 (非加值型): Local Tax = 1785, Online Tax = 1785


===== 迴圈第 3 次 =====
Test 3 (加值型): Local Tax = -2139, Online Tax = -2139
Test 3 (非加值型): Local Tax = 668, Online Tax = 668


===== 迴圈第 4 次 =====
Test 4 (加值型): Local Tax = 239, Online Tax = 239
Test 4 (非加值型): Local Tax = 631, Online Tax = 631


===== 迴圈第 5 次 =====
Test 5 (加值型): Local Tax = -492, Online Tax = -492
Test 5 (非加值型): Local Tax = 207, Online Tax = 207


===== 迴圈第 6 次 =====
Test 6 (加值型): Local Tax = 4776, Online Tax = 4776
Test 6 (非加值型): Local Tax = 6462, Online Tax = 6462


===== 迴圈第 7 次 =====
Test 7 (加值型): Local Tax = -410, Online Tax = -410
Test 7 (非加值型): Local Tax = 25397, Online Tax = 25397


===== 迴圈第 8 次 =====
Test 8 (加值型): Local Tax = -1453, Online Tax = -1453
Test 8 (非加值型): Local Tax = 15525, Online Tax = 15525


===== 迴圈第 9 次 ==

### REVERIFY MISMATCHES PARAMS

Regarding the Business Tax validation across both Value-Added (VAT) and Non-Value-Added regimes, we conducted 100 test runs for each category, achieving a 98.5% overall match rate. For the minor discrepancies flagged during the automated Selenium process (e.g., Tests 17, 51, and 84), we have performed a manual cross-verification by re-entering the exact parameters into the official portal. The manual results confirm that our local engine’s calculations are, in fact, fully consistent with the official government tool, suggesting that the previously reported mismatches were likely artifacts of the automation script's data retrieval process rather than logic errors.


Portal URL : [https://www.etax.nat.gov.tw/etwmain/etw158w/40]